In [2]:
from datascience import *

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn import tree
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import graphviz
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

from sklearn.externals import joblib

In [4]:
# Load text data
data = Table.read_table('data_stopwords_removed_URLs_replaced.csv', sep=',')
data

text,label
rt USER_MENTION breakfast. #kingsman table manner scene ...,0
rt USER_MENTION #junkai # kingsman 2 : golden circle_,1
review: kingsman partner statesman located kentucky ever ...,0
--kingsman spoilers-- merlin's end greatest character de ...,0
24. kingsman. el c rculo de oro. URL,0
ye olde cock tavern fleet st london. photo veronik. | #a ...,0
infilmercials - kingsman: secret service: URL via USER_M ...,0
gust un video de USER_MENTION URL kingsman 2 director st ...,0
rt USER_MENTION halle berry pedro pascal tease romance k ...,0
oh kingsman 2 dumb lit,0


In [14]:
# Split data into data_train and data_new
data_train = data.take(range(200)) # First 200 rows
data_new = data.take(range(200, data.num_rows))

# Remove the `label` column of data_new
data_new = data_new.select(0)

In [25]:
data_train

text,label
rt USER_MENTION breakfast. #kingsman table manner scene ...,0
rt USER_MENTION #junkai # kingsman 2 : golden circle_,1
review: kingsman partner statesman located kentucky ever ...,0
--kingsman spoilers-- merlin's end greatest character de ...,0
24. kingsman. el c rculo de oro. URL,0
ye olde cock tavern fleet st london. photo veronik. | #a ...,0
infilmercials - kingsman: secret service: URL via USER_M ...,0
gust un video de USER_MENTION URL kingsman 2 director st ...,0
rt USER_MENTION halle berry pedro pascal tease romance k ...,0
oh kingsman 2 dumb lit,0


In [26]:
data_new

text
USER_MENTION USER_MENTION USER_MENTION USER_MENTION i'm ...
rt USER_MENTION m telling go see #kingsman USER_MENTION ...
rt USER_MENTION m telling go see #kingsman USER_MENTION ...
rt USER_MENTION pug first kingsman movie attended premie ...
rt USER_MENTION training wheels. off. #kingsman: golden ...
rt USER_MENTION maybe ve heard them? 5 reasons #kingsman ...
URL : kingsman 2 golden circle: channing tatum sex toy c ...
also say writing kingsman secret service way better king ...
rt USER_MENTION 'kingsman: golden circle' theatres lets ...
'kingsman: golden circle' building bigger better james b ...


In [16]:
# Prepare data_train
data_train_X = list(data_train['text'])
data_train_y = list(data_train['label'])

# Train model on data_train
clf = Pipeline([('vect', CountVectorizer()), 
                ('clf', tree.DecisionTreeClassifier(max_depth=3))])
clf.fit(data_train_X, data_train_y)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...      min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))])

In [17]:
# Save the trained model
from sklearn.externals import joblib
joblib.dump(clf, 'model.pkl')

['model.pkl']

In [21]:
# Load the saved model
clf2 = joblib.load('model.pkl')

# Predict on data_new
predicted = clf2.predict(data_new['text'])
predicted

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])

In [24]:
# Append predictions to data_new
data_new_pred = data_new.with_columns('predicted', predicted)
# Save
data_new_pred.to_csv('data_new_pred.csv')

In [23]:
data_new_pred

text,predicted
USER_MENTION USER_MENTION USER_MENTION USER_MENTION i'm ...,0
rt USER_MENTION m telling go see #kingsman USER_MENTION ...,0
rt USER_MENTION m telling go see #kingsman USER_MENTION ...,0
rt USER_MENTION pug first kingsman movie attended premie ...,0
rt USER_MENTION training wheels. off. #kingsman: golden ...,0
rt USER_MENTION maybe ve heard them? 5 reasons #kingsman ...,0
URL : kingsman 2 golden circle: channing tatum sex toy c ...,0
also say writing kingsman secret service way better king ...,0
rt USER_MENTION 'kingsman: golden circle' theatres lets ...,0
'kingsman: golden circle' building bigger better james b ...,0


In [28]:
# If saving a classifier instance instead of Pipeline
count_vect = CountVectorizer()
data_train_X_count = count_vect.fit_transform(data_train_X)

clf3 = tree.DecisionTreeClassifier(max_depth=3)
clf3.fit(data_train_X_count, data_train_y)
joblib.dump(clf3, 'model_clf.pkl')

['model_clf.pkl']

In [29]:
# Load model_clf.pkl
clf4 = joblib.load('model_clf.pkl')
clf4.predict(data_new['text'])

ValueError: could not convert string to float: "USER_MENTION USER_MENTION USER_MENTION USER_MENTION i'm going see tomorrow. #kingsman"

In [37]:
# Example 2: Iris data
iris = Table.read_table('iris.csv')
# Random shuffle
iris = iris.sample(iris.num_rows)

iris_train = iris.take(range(100))
iris_new = iris.take(range(100, iris.num_rows))
iris_new = iris_new.select(0,1,2,3)

In [38]:
iris_train

Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
5.9,3,5.1,1.8,virginica
6.3,2.3,4.4,1.3,versicolor
6.3,3.3,6,2.5,virginica
5,3.6,1.4,0.2,setosa
4.5,2.3,1.3,0.3,setosa
4.9,3.1,1.5,0.2,setosa
4.8,3.1,1.6,0.2,setosa
5.8,2.7,5.1,1.9,virginica
6.4,3.2,4.5,1.5,versicolor
5.7,4.4,1.5,0.4,setosa


In [39]:
iris_new

Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
6,2.7,5.1,1.6
6.8,3.2,5.9,2.3
4.4,2.9,1.4,0.2
6.2,3.4,5.4,2.3
5.1,3.8,1.5,0.3
4.8,3,1.4,0.1
6.7,3.3,5.7,2.5
4.4,3,1.3,0.2
6.7,3.1,4.4,1.4
5.1,3.5,1.4,0.3


In [40]:
# Prepare data
iris_train_X = iris_train.select(0,1,2,3).to_df().as_matrix()
iris_train_y = iris_train.select(4).to_df().as_matrix()

# Train and save
clf5 = tree.DecisionTreeClassifier(max_depth=3)
clf5.fit(iris_train_X, iris_train_y)
joblib.dump(clf5, 'model_iris.pkl')

['model_iris.pkl']

In [41]:
# Load and predict
clf6 = joblib.load('model_iris.pkl')

iris_new_X = iris_new.to_df().as_matrix()
predicted = clf6.predict(iris_new_X)

In [42]:
predicted

array(['versicolor', 'virginica', 'setosa', 'virginica', 'setosa',
       'setosa', 'virginica', 'setosa', 'versicolor', 'setosa', 'setosa',
       'versicolor', 'virginica', 'virginica', 'virginica', 'setosa',
       'versicolor', 'setosa', 'setosa', 'versicolor', 'virginica',
       'versicolor', 'setosa', 'setosa', 'virginica', 'setosa',
       'virginica', 'versicolor', 'virginica', 'virginica', 'virginica',
       'versicolor', 'setosa', 'virginica', 'setosa', 'setosa', 'setosa',
       'virginica', 'setosa', 'virginica', 'virginica', 'virginica',
       'virginica', 'setosa', 'versicolor', 'versicolor', 'versicolor',
       'virginica', 'virginica', 'setosa'], dtype=object)